In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from math import nan

pd.set_option('display.max_columns', None)


# Aufbereitung

## Monatsbericht


In [20]:
def prepare_monatsbericht(path_origin, path_aim):
    df_1082 = pd.read_csv(path_origin, skiprows =7,
            skipfooter = 4, sep = ';'
            )
    #Adapt column names
    df_1082.columns = ['Steller', 'Produkt', 'Betriebe', 'Betriebe_ep', 'Beschäftigte',
       'Beschäftigte_ep', 'Geleistete Arbeitsstunden', 'Geleistete Arbeitsstunden_ep',
       'Bruttolohn- und -gehaltssumme', 'Bruttolohn- und -gehaltssumme_ep', 'Umsatz', 'Umsatz_ep',
       'Inlandsumsatz', 'Inlandsumsatz_ep', 'Auslandsumsatz', 'Auslandsumsatz_ep',
       'Auslandsumsatz mit der Eurozone', 'Auslandsumsatz mit der Eurozone_ep',
       'Auslandsumsatz mit dem sonstigen Ausland', 'Auslandsumsatz mit dem sonstigen Ausland_ep']
    different_steller = df_1082['Steller'].unique().tolist()
    num_diff_steller = len(different_steller)-12-7-1
    #Add date
    df_1082['month']=0
    for p in range(7):
        k = 2 + (1+(num_diff_steller +1)*12)*p
        for month in range(1,13): 
            for i in range(1,(num_diff_steller +1)):
                df_1082.iloc[k+i+(month-1)*(num_diff_steller +1), -1] = month
    for p in range(7):
        k = 2 + (1+(num_diff_steller +1)*12)*p
        for month in range(0,12): 
            df_1082 =  df_1082.drop(k + month * (num_diff_steller +1),axis=0)
    df_1082['year']=0
    for k in range(7):
        for i in range(1 + 12 * num_diff_steller):
            df_1082.iloc[1 + k* (1 + 12 * num_diff_steller) + i, -1] = k + 2019
    for k in range(7):
        df_1082 =df_1082.drop(1 + k*(1 + 12 * (num_diff_steller+1)),axis = 0)
    df_1082 =df_1082.drop(0,axis = 0)
    df_1082['day']=1
    df_1082['date']=pd.to_datetime(df_1082[['year', 'month', 'day']])

    df_1082 = df_1082.drop('month', axis = 1)
    df_1082 = df_1082.drop('year', axis = 1)
    df_1082 = df_1082.drop('day', axis = 1)
    #Include multiplicities
    for col in ['Betriebe', 'Beschäftigte', 'Geleistete Arbeitsstunden', 'Bruttolohn- und -gehaltssumme', 
            'Inlandsumsatz', 'Auslandsumsatz', 'Auslandsumsatz mit der Eurozone', 'Umsatz',
            'Auslandsumsatz mit dem sonstigen Ausland']:
        df_1082[col]=df_1082[col].replace('...', nan)
        df_1082[col]=df_1082[col].astype('float64')
    df_1082['Geleistete Arbeitsstunden']=df_1082['Geleistete Arbeitsstunden'] * 1000
    df_1082['Bruttolohn- und -gehaltssumme']=df_1082['Bruttolohn- und -gehaltssumme'] * 1000
    df_1082['Umsatz']=df_1082['Umsatz'] * 1000
    df_1082['Inlandsumsatz']=df_1082['Inlandsumsatz'] * 1000
    df_1082['Auslandsumsatz']=df_1082['Auslandsumsatz'] * 1000
    df_1082['Auslandsumsatz mit der Eurozone']=df_1082['Auslandsumsatz mit der Eurozone'] * 1000
    df_1082['Auslandsumsatz mit dem sonstigen Ausland']=df_1082['Auslandsumsatz mit dem sonstigen Ausland'] * 1000
    #Divide df in a df for each different Steller
    different_steller = df_1082['Steller'].unique().tolist()
    df_separated = []
    for steller in different_steller:
        mask = (df_1082['Steller'] == steller)
        df_help = df_1082.loc[mask,:]
        df_help = df_help.drop('Steller', axis=1)
        df_help = df_help.drop('Produkt', axis=1)
        new_columns = []
        for col in df_help.columns.values[:-1]:
            new_columns.append(col + '_' + steller)
        new_columns.append('date')
        df_help.columns = new_columns
        df_help = df_help.set_index('date')
        df_separated.append(df_help)
    df_1082_prepared = pd.concat(df_separated, axis = 1)
    df_1082_prepared.to_csv(path_aim)

In [22]:
path_origin = '../data/raw/1082ff_monatsbericht_suesswaren_schoko_kaffee.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'
prepare_monatsbericht(path_origin, path_aim)

C:\Users\judit\AppData\Local\Temp\ipykernel_25848\1452552771.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df_1082 = pd.read_csv(path_origin, skiprows =7,


In [23]:
path_origin = '../data/raw/1082_monatsbericht_suesswaren_timeseries.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'
prepare_monatsbericht(path_origin, path_aim)

C:\Users\judit\AppData\Local\Temp\ipykernel_25848\1452552771.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df_1082 = pd.read_csv(path_origin, skiprows =7,


## Einfuhr und Erzeugerpreise

In [ ]:
def prepare_GP(path_origin, path_aim, praefix = ''):
    df= pd.read_csv(path_origin, 
                skiprows =6,
                skipfooter = 3, sep = ';', decimal = ','
                )
    for i in range(3, 171, 2):
        zahl = str(i)
        df = df.drop('Unnamed: '+zahl, axis=1)
    df = df.transpose()
    columns = df.iloc[1,:]
    columns.values[0] = 'Month'
    for i in range(1,len(columns)):
        columns.values[i] = praefix + '_' + columns.values[i]
    df.columns = columns
    df = df.drop('Unnamed: 0', axis = 0)
    df = df.drop('Unnamed: 1', axis = 0)
    list_year = []
    for i in range(7):
        list_year = list_year + np.full(12,i+2019).tolist()
    df['Year'] = list_year
    df['Day']=1
    df['Month'] = df['Month'].replace({'Januar':1, 'Februar':2, 'März': 3, 'April':4, 'Mai':5, 
                                   'Juni': 6, 'Juli': 7, 'August': 8, 'September': 9, 'Oktober': 10, 'November': 11, 'Dezember': 12})

    df['date']=pd.to_datetime(df[['Year', 'Month', 'Day']])
    df = df.drop('Month', axis = 1)
    df = df.drop('Year', axis = 1)
    df = df.drop('Day', axis = 1)
    df = df.set_index('date')
    for col in df.columns:
        df[col]=df[col].astype('float64')
    df.to_csv(path_aim)

In [31]:
path_origin = '../data/raw/1082ff_erzeugerpreise_suesswaren_schoko_kaffee.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'
prepare_GP(path_origin, path_aim, 'ErzPr')

C:\Users\judit\AppData\Local\Temp\ipykernel_25848\3763938883.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df= pd.read_csv(path_origin,
C:\Users\judit\AppData\Local\Temp\ipykernel_25848\3763938883.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Month'] = df['Month'].replace({'Januar':1, 'Februar':2, 'März': 3, 'April':4, 'Mai':5,


Unnamed: 1,ErzPr_Nahrungsmittel und Futtermittel,ErzPr_ Sonstige Nahrungsmittel (ohne Getränke),ErzPr_ Süßwaren (ohne Dauerbackwaren),ErzPr_ Schokoladen und andere Süßwaren,"ErzPr_ Schokolade u.a. kakaoh. Lebensm.zub.,in Verp.>2kg","ErzPr_ Schokolade u.a. kakaoh. Leb.m.zuber.,in Verp.<=2kg",ErzPr_ Süßwaren oh. Kakaogeh. (einschl.weißer Schokolade),"ErzPr_ Kaffee und Tee, Kaffee-Ersatz","ErzPr_ Kaffee und Tee, Kaffee-Ersatz","ErzPr_ Kaffee, entkoffeiniert oder geröstet","ErzPr_ Kaffeemittel m. -geh.,Auszüge,Essenz.,Konzentrate","ErzPr_ Grüner Tee (n. ferment.), schwarzer Tee (ferment.)",ErzPr_ Kräutertees
date,,,,,,,,,,,,,
2019-01-01,95.2,95.9,97.0,97.0,95.6,97.6,96.5,98.4,98.4,98.1,101.6,95.9,95.8
2019-02-01,95.2,96.1,97.0,97.0,96.1,97.6,96.5,98.2,98.2,97.8,101.5,95.3,97.2
2019-03-01,95.1,96.1,96.9,96.9,95.9,97.4,96.5,98.3,98.3,97.9,101.5,95.3,97.2
2019-04-01,96.6,96.1,97.0,97.0,97.4,97.5,96.5,97.9,97.9,97.3,101.5,95.3,97.2
2019-05-01,97.1,96.0,97.0,97.0,96.2,97.5,96.5,97.6,97.6,96.8,101.5,95.3,97.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-01,135.8,143.2,163.6,163.6,265.5,159.3,126.4,166.5,166.5,179.6,154.1,109.4,123.9
2025-09-01,135.7,143.5,164.4,164.4,249.7,163.0,126.4,165.9,165.9,178.6,154.9,108.6,123.0
2025-10-01,134.0,142.2,160.9,160.9,221.1,162.3,126.4,163.0,163.0,174.4,154.9,108.9,123.4


In [32]:
path_origin = '../data/raw/1082_1083_einfuhrpreise_suesswarenundkaffee.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'
prepare_GP(path_origin, path_aim, 'EinfPr')

C:\Users\judit\AppData\Local\Temp\ipykernel_25848\3763938883.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df= pd.read_csv(path_origin,
C:\Users\judit\AppData\Local\Temp\ipykernel_25848\3763938883.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Month'] = df['Month'].replace({'Januar':1, 'Februar':2, 'März': 3, 'April':4, 'Mai':5,


Unnamed: 1,EinfPr_Nahrungsmittel und Futtermittel,EinfPr_ Sonstige Nahrungsmittel (ohne Getränke),EinfPr_ Süßwaren (ohne Dauerbackwaren),"EinfPr_ Kaffee und Tee, Kaffee-Ersatz"
date,,,,
2019-01-01,95.0,98.9,102.4,99.0
2019-02-01,95.2,98.9,102.3,97.9
2019-03-01,95.2,98.9,102.5,97.8
2019-04-01,96.0,98.8,102.3,98.3
2019-05-01,96.2,98.8,102.4,98.3
...,...,...,...,...
2025-08-01,143.9,172.0,235.0,175.1
2025-09-01,143.6,168.8,226.6,175.1
2025-10-01,142.1,165.6,216.4,175.5


### VPI

In [33]:
path_origin = '../data/raw/01183ff_vpi_suesswaren_schoko_kaffee.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'

In [34]:
df= pd.read_csv(path_origin, 
                skiprows =6,
                skipfooter = 4, sep = ';', decimal = ','
                )
for i in range(3, 171, 2):
    zahl = str(i)
    df = df.drop('Unnamed: '+zahl, axis=1)
df = df.transpose()
praefix = 'VPI'
columns = df.iloc[1,:]
columns.values[1] = 'Month'
for i in range(2,len(columns)):
    columns.values[i] = praefix + '_' + columns.values[i]
df.columns = columns
df = df.drop('Verbraucherpreisindex (2020=100)', axis = 0)
df = df.drop('Unnamed: 1', axis = 0)
df = df.drop(nan, axis = 1)
list_year = []
for i in range(7):
    list_year = list_year + np.full(12,i+2019).tolist()
df['Year'] = list_year
df['Day']=1
df['Month'] = df['Month'].replace({'Januar':1, 'Februar':2, 'März': 3, 'April':4, 'Mai':5, 
                                   'Juni': 6, 'Juli': 7, 'August': 8, 'September': 9, 'Oktober': 10, 'November': 11, 'Dezember': 12})

df['date']=pd.to_datetime(df[['Year', 'Month', 'Day']])
df = df.drop('Month', axis = 1)
df = df.drop('Year', axis = 1)
df = df.drop('Day', axis = 1)
df = df.set_index('date')
for col in df.columns:
    df[col]=df[col].astype('float64')
df.to_csv(path_aim)

C:\Users\judit\AppData\Local\Temp\ipykernel_25848\1458979757.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df= pd.read_csv(path_origin,
C:\Users\judit\AppData\Local\Temp\ipykernel_25848\1458979757.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Month'] = df['Month'].replace({'Januar':1, 'Februar':2, 'März': 3, 'April':4, 'Mai':5,


### A1014


In [130]:
path_origin = '../data/raw/A1014_17_18_monprod_suesswarenequi_kaffee.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'

In [131]:
df= pd.read_csv(path_origin, 
                skiprows =4,
                skipfooter = 4, sep = ';'
                )
list_month = [nan, nan]
for i in range(1,13):
    list_month = list_month + np.full(22,i).tolist()
df.loc[0,:]=list_month
list_year = [nan, nan, nan]
for i in range(2019,2026):
    list_year = list_year + np.full(4,i).tolist()
df.loc[:,'Deutschland']=list_year
for i in range(7):
    df = df.drop(3+ i *4)
df.loc[1, 'Deutschland'] = 1
df.loc[1, 'Unnamed: 1'] = 1
# Ab hier monatsspezifisch
df_monate = []
for i in range(1,13):
    df_help = df.loc[:, (df.loc[0, :] == i) | (df.loc[1,:] == 1)]
    df_help['Month'] = i
    df_help = df_help.drop(0)
    columns = ['Year', 'Name', 'Wert der zum Absatz bestimmten Produktion', 
           'Wert der zum Absatz bestimmten Produktion_ep', 
           'Gewicht der zum Absatz bestimmten Produktion_t',
           'Gewicht der zum Absatz bestimmten Produktion_t_ep',
           'Gewicht der zum Absatz bestimmten Produktion_kg', 
           'Gewicht der zum Absatz bestimmten Produktion_kg_ep', 
           'Menge der zum Absatz bestimmten Produktion_stueck', 
           'Menge der zum Absatz bestimmten Produktion_stueck_ep', 
           'Menge der zum Absatz bestimmten Produktion_jewME', 
           'Menge der zum Absatz bestimmten Produktion_jewME_ep',
           'Unternehmen mit zum Absatz bestimmter Produktion',
           'Unternehmen mit zum Absatz bestimmter Produktion_ep',
           'Gewicht der Gesamtproduktion_t',
           'Gewicht der Gesamtproduktion_t_ep', 
           'Gewicht der Gesamtproduktion_kg',
           'Gewicht der Gesamtproduktion_kg_ep', 
           'Menge der Gesamtproduktion_stueck', 
           'Menge der Gesamtproduktion_stueck_ep', 
           'Menge der Gesamtproduktion_jewME',
           'Menge der Gesamtproduktion_jewME_ep', 
           'Unternehmen mit Produktion',
           'Unternehmen mit Produktion_ep', 
           'Month'
           ]
    df_help.columns = columns
    df_help = df_help.drop(1)
    vielfachheiten = df_help.loc[2, :]
    df_help = df_help.drop(2)
    for col in ['Wert der zum Absatz bestimmten Produktion', 
           'Gewicht der zum Absatz bestimmten Produktion_t',
           'Gewicht der zum Absatz bestimmten Produktion_kg', 
           'Menge der zum Absatz bestimmten Produktion_stueck',  
           'Menge der zum Absatz bestimmten Produktion_jewME', 
           'Unternehmen mit zum Absatz bestimmter Produktion',
           'Gewicht der Gesamtproduktion_t',
           'Gewicht der Gesamtproduktion_kg',
           'Menge der Gesamtproduktion_stueck', 
           'Menge der Gesamtproduktion_jewME',
           'Unternehmen mit Produktion']:
        df_help[col]=df_help[col].replace('...', nan)
        df_help[col]=df_help[col].replace('-', nan)
        df_help[col]=df_help[col].astype('float64')
    df_help['Year']=df_help['Year'].astype('int64')
    df_help['Month']=df_help['Month'].astype('int64')
    df_help['Wert der zum Absatz bestimmten Produktion']=df_help['Wert der zum Absatz bestimmten Produktion']*1000
    df_help['Day']=1
    df_help['date']=pd.to_datetime(df_help[['Year', 'Month', 'Day']])

    df_help = df_help.drop('Month', axis = 1)
    df_help = df_help.drop('Year', axis = 1)
    df_help = df_help.drop('Day', axis = 1)
    df_help = df_help.set_index('date')
    df_monate.append(df_help)

df_end = pd.concat(df_monate, axis = 0)
different_steller = df_end['Name'].unique().tolist()
df_separated = []
for steller in different_steller:
    mask = (df_end['Name'] == steller)
    df_help = df_end.loc[mask,:]
    df_help = df_help.drop('Name', axis=1)
    new_columns = []
    for col in df_help.columns.values:
        new_columns.append(col + '_' + steller)
    #new_columns.append('date')
    df_help.columns = new_columns
    #df_help = df_help.set_index('date')
    df_separated.append(df_help)
df_endagain = pd.concat(df_separated, axis = 1)
df_endagain.to_csv(path_aim)

C:\Users\judit\AppData\Local\Temp\ipykernel_30068\1850794607.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df= pd.read_csv(path_origin,
C:\Users\judit\AppData\Local\Temp\ipykernel_30068\1850794607.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_help['Month'] = i
C:\Users\judit\AppData\Local\Temp\ipykernel_30068\1850794607.py:63: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_help[col]=df_

### global prices


In [46]:
def prepare_globalprice(path_origin, path_aim):
    df= pd.read_csv(path_origin, 
                skiprows =0,
                skipfooter = 0, sep = ','#, index_col = 'observation_date'
                )
    df.columns = ['date', df.columns[1]]
    df = df.set_index('date')
    df.to_csv(path_aim)

In [47]:
path_origin = '../data/raw/globalcacaoprice_fred.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'

prepare_globalprice(path_origin, path_aim)

In [48]:
path_origin = '../data/raw/globalcoffeerobustaprice_fred.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'

prepare_globalprice(path_origin, path_aim)

### 0118ff waegung vpi

In [49]:
path_origin = '../data/0118ff_waegung_vpi_suesswaren_schoko_kaffee.csv'
path_aim = '../data/processed/' + path_origin[12:-4] + '_processed.csv'


In [ ]:
df= pd.read_csv(path_origin, 
                skiprows =7,
                skipfooter = 3, sep = ';'
                )
df = df.drop('Unnamed: 0', axis = 1)
df = df.drop('Unnamed: 3', axis = 1)
df = df.set_index('Unnamed: 1')
df = df.transpose()
df.to_csv(path_aim)

C:\Users\judit\AppData\Local\Temp\ipykernel_25848\2721576748.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df= pd.read_csv(path_origin,
